### Importing necessary packages

In [146]:
import pandas as pd
import numpy as np
import requests
import time
from opencage.geocoder import OpenCageGeocode
from math import radians, sin, cos, sqrt, atan2
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from scipy.spatial import distance_matrix

In [147]:
# Load the Ebonyi state dataset
Lagos_data = pd.read_csv(r'C:\Users\User\Downloads\Data Azure\Lagos_crosschecked.csv')

In [148]:
Lagos_data

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File
0,LAGOS,AGEGE,ISALE/IDIMANGORO,24-01-01-001,"ST. JOSEPH SECONDARY SCHOOL, MANGORO",120,744,True,-1,False,True,False,False,UNKNOWN,0,0,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
1,LAGOS,AGEGE,ISALE/IDIMANGORO,24-01-01-002,"15, SHOMIDE STREET",111,716,True,-1,False,False,False,False,UNKNOWN,0,0,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
2,LAGOS,AGEGE,ISALE/IDIMANGORO,24-01-01-004,"8, SEBIOTIMO STREET",108,843,True,-1,False,True,False,False,UNKNOWN,31,57,11,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
3,LAGOS,AGEGE,ISALE/IDIMANGORO,24-01-01-005,"6, SEBIOTIMO STREET",114,731,True,-1,False,True,False,False,UNKNOWN,28,66,5,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
4,LAGOS,AGEGE,ISALE/IDIMANGORO,24-01-01-006,WITHIN OLABI SQUARE,80,539,True,-1,False,False,False,False,UNKNOWN,24,28,16,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11906,LAGOS,IKORODU,BAIYEKU/ORETA,24-12-11-029,"OPEN SPACE AT OKUNUGA ROUND ABOUT, OFFIN-ILE",75,138,True,-1,False,False,False,False,UNKNOWN,47,22,3,0,https://docs.inecelectionresults.net/elections...
11907,LAGOS,OJO,OKOKOMAIKO,24-17-02-032,OPEN SPACE INFRONT OF OKOKO TOWN HALL,133,276,True,-1,False,False,False,False,UNKNOWN,71,51,8,1,https://docs.inecelectionresults.net/elections...
11908,LAGOS,OJO,OKOKOMAIKO,24-17-02-033,OPEN SPACE IN FRONT OF NO 1 OLANREWAJU STR.,119,198,True,-1,False,False,False,False,UNKNOWN,35,78,2,1,https://docs.inecelectionresults.net/elections...
11909,LAGOS,OJO,OKOKOMAIKO,24-17-02-034,"OPEN SPACE AT NO 6, AGOMALE STREET",59,107,True,-1,False,False,False,False,UNKNOWN,46,5,3,0,https://docs.inecelectionresults.net/elections...


In [149]:
def geocode_address(address, api_key, retries=5):
    url = f'https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}'
    for i in range(retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                if data['results']:
                    return data['results'][0]['geometry']['lat'], data['results'][0]['geometry']['lng']
            time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Retrying ({i+1}/{retries})...")
            time.sleep(1)
    return None, None

# Use your new OpenCage API key
api_key = '641c20f29104452cbd87e70c9a379aa8'

# Load the CSV file
Lagos_data = pd.read_csv(r'C:\Users\User\Downloads\Data Azure\Lagos_crosschecked.csv')

# Combine relevant columns to form the address
Lagos_data['address'] = Lagos_data['PU-Name'] + ', ' + Lagos_data['Ward'] + ', ' + Lagos_data['LGA'] + ', ' + 'Lagos, Nigeria'

# Geocode each polling unit
latitudes = []
longitudes = []

for address in Lagos_data['address']:
    lat, lon = geocode_address(address, api_key)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(1)  # to respect API rate limits

# Add latitude and longitude to the DataFrame
Lagos_data['latitude'] = latitudes
Lagos_data['longitude'] = longitudes

# Drop rows with missing latitude or longitude
Lagos_data.dropna(subset=['latitude', 'longitude'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = r'C:\Users\User\Downloads\Data Azure\Lagos_crosschecked_geocoded.csv'
Lagos_data.to_csv(output_file, index=False)
print(f"Geocoded data saved to {output_file}")


KeyboardInterrupt: 

In [133]:
df = pd.read_csv(output_file_path).shape

In [134]:
df

(8, 23)

In [124]:
df.columns

Index(['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name', 'Accredited_Voters',
       'Registered_Voters', 'Results_Found', 'Transcription_Count',
       'Result_Sheet_Stamped', 'Result_Sheet_Corrected',
       'Result_Sheet_Invalid', 'Result_Sheet_Unclear', 'Result_Sheet_Unsigned',
       'APC', 'LP', 'PDP', 'NNPP', 'Results_File', 'address', 'latitude',
       'longitude', 'cluster'],
      dtype='object')

In [129]:
# subset the dataset
data = df.loc[:, ['State','LGA','Ward','PU-Code','PU-Name','APC','LP','PDP','NNPP','latitude','longitude']].copy()
data

,State,LGA,Ward,PU-Code,PU-Name,APC,LP,PDP,NNPP,latitude,longitude
0,EBONYI,AFIKPO NORTH,EZEKE AMASIRI,11-02-03-011,AMEBO NDUKWE VILLAGE SQUARE,11,155,18,0,5.89312,7.93735
1,EBONYI,EBONYI,ENYIBICHIRI I,11-04-07-006,OGUZORONWEYA PRIMARY SCHOOL I,22,115,28,0,6.52184,8.12984
2,EBONYI,EZZA NORTH,NKOMORO,11-05-05-010,AMENYI,4,55,3,1,6.27078,7.97127
3,EBONYI,IKWO,NOYO ALIKE,11-07-20-005,C.S NDIAGU NOYO,10,115,29,4,6.07451,8.10060
4,EBONYI,ISHIELU,EZZAGU II (NKOMORO),11-08-08-026,ONUNGANGBO IGWE PLAY GROUND,2,25,3,2,6.42714,7.83583
5,EBONYI,IZZI,AGBAJA MGBO,11-10-01-009,AGUBATA VILLAGE HALL,29,112,14,0,6.58597,8.05301
6,EBONYI,ONICHA,AGBABOR-ISU,11-13-02-007,UFUOBA PLAY GROUND UFUOBA,10,76,11,0,6.10945,7.81753
7,EBONYI,ABAKALIKI,ABAKPA,11-01-01-013,LOT 1B INTERNATIONAL MKT,0,4,0,0,6.32485,8.11368


In [127]:
#check for the percentage of null values in each columns
(data.isnull().sum()/len(data)*100)
# The percentage of null values in both lat ang lon is 100 we need to drop those columns

State        0.0
LGA          0.0
Ward         0.0
PU-Code      0.0
PU-Name      0.0
APC          0.0
LP           0.0
PDP          0.0
NNPP         0.0
latitude     0.0
longitude    0.0
dtype: float64

In [67]:
# Assuming you've read your CSV into a DataFrame df
df = pd.read_csv(r'C:\Users\User\Downloads\Data Azure\Ebonyi_crosschecked_geocoded.csv')

# Drop rows with any NaN/null values
df.dropna(inplace=True)

# Now df contains the DataFrame with null values dropped


In [68]:
print(df)

       State        LGA               Ward       PU-Code  \
0     EBONYI  ABAKALIKI             ABAKPA  11-01-01-002   
1     EBONYI  ABAKALIKI             ABAKPA  11-01-01-003   
2     EBONYI  ABAKALIKI             ABAKPA  11-01-01-004   
3     EBONYI  ABAKALIKI             ABAKPA  11-01-01-006   
4     EBONYI  ABAKALIKI             ABAKPA  11-01-01-007   
...      ...        ...                ...           ...   
2495  EBONYI    ISHIELU          EZILLO II  11-08-06-013   
2496  EBONYI    ISHIELU          EZILLO II  11-08-06-014   
2497  EBONYI    ISHIELU               AGBA  11-08-01-015   
2498  EBONYI    ISHIELU      AZUINYABA "A"  11-08-03-006   
2499  EBONYI    ISHIELU  EZZAGU I (OGBOJI)  11-08-07-013   

                            PU-Name  Accredited_Voters  Registered_Voters  \
0                ADAZI-ENU HALL  II                150                750   
1             CO-OPERATIVE SQUARE I                257               1045   
2            CO-OPERATIVE SQUARE II             

Clustered data saved to C:\Users\User\Downloads\Data Azure\Ebonyi_with_clusters.csv


In [92]:

def calculate_outlier_score(polling_unit, neighbours, party_columns):
    scores = {}
    for party in party_columns:
        neighbour_votes = df.loc[neighbours, party].values
        if len(neighbour_votes) > 0:
            mean_neighbour_votes = np.mean(neighbour_votes)
            scores[party] = abs(polling_unit[party] - mean_neighbour_votes)
        else:
            scores[party] = 0  # If no neighbours, score is 0
    return scores

# Define the columns that contain the votes for each party
party_columns = ['APC', 'LP', 'PDP', 'NNPP'] 

# Add outlier scores to the dataset
df['outlier_scores'] = df.apply(lambda row: calculate_outlier_score(row, row['cluster'], party_columns), axis=1)

# Expand outlier scores into separate columns
outlier_scores_df = pd.DataFrame(df['outlier_scores'].tolist(), index=df.index)

# Remove existing outlier score columns if they exist
for party in party_columns:
    outlier_col = f"{party}_outlier_score"
    if outlier_col in df.columns:
        df.drop(columns=[outlier_col], inplace=True)

# Join the new outlier score columns
df = df.join(outlier_scores_df.add_suffix('_outlier_score'))

# Save the updated dataset as an Excel file
output_file = 'Ebonyi_Polling_Units_Sorted_By_Outlier_Scores.xlsx'
df.to_excel(output_file, index=False)
print(f"Outlier scores saved to {output_file}")


KeyError: 'cluster'